In [1]:
from analyst.quantitative_analyst.quantitative_analyst import QuantitativeAnalyst
from symbol.symbol import Symbol

from infrastructure.databases.company.postgre_manager.company_data_manager import CompanyDataManager

symbol = "NVDA"  # Replace with user input as needed
adapter = CompanyDataManager()
storage = Symbol(adapter, symbol)
df = storage.daily_timeseries

2025-09-21 14:39:31,523 - infrastructure.databases.company.postgre_manager.company_data_manager - INFO - company_data_manager.py:459 - Loaded 1 filtered rows from company_fundamentals
2025-09-21 14:39:31,539 - infrastructure.databases.company.postgre_manager.company_data_manager - INFO - company_data_manager.py:459 - Loaded 20 filtered rows from balance_sheet_annual
2025-09-21 14:39:31,544 - db_load_from_db_runner - INFO - symbol.py:189 - Loaded table 'balance_sheet_annual' with 20 rows.
2025-09-21 14:39:31,556 - infrastructure.databases.company.postgre_manager.company_data_manager - INFO - company_data_manager.py:459 - Loaded 20 filtered rows from cash_flow_annual
2025-09-21 14:39:31,560 - db_load_from_db_runner - INFO - symbol.py:189 - Loaded table 'cash_flow_annual' with 20 rows.
2025-09-21 14:39:31,566 - infrastructure.databases.company.postgre_manager.company_data_manager - INFO - company_data_manager.py:459 - Loaded 27 filtered rows from earnings_annual
2025-09-21 14:39:31,569 - 

In [4]:
import logging
import pandas as pd
from typing import List, Optional, Dict


def _find_quarterly_tables() -> List[pd.DataFrame]:
    """Return a list of DataFrames corresponding to quarterly statements.

    The ``Symbol`` instance exposes each database table as an
    attribute whose name matches the underlying table name.  In
    different parts of the codebase the word *quarterly* is
    sometimes misspelled as ``quaterly``.  To be resilient to both
    conventions, this method looks for any attribute on ``self.symbol``
    that contains ``income_statement``, ``balance_sheet`` or ``cash_flow``
    and ends with ``quarterly`` or ``quaterly``.  Tables that are
    present and non‑empty are returned.
    """
    tables: List[pd.DataFrame] = []
    # Possible stems for the three statement types
    stems = ["income_statement", "balance_sheet", "cash_flow"]
    for attr_name in dir(symbol):
        # Skip private attributes
        if attr_name.startswith("_"):
            continue
        lower_name = attr_name.lower()
        # Only consider attributes that include one of the stems and end with the quarterly suffix
        if any(stem in lower_name for stem in stems) and (
            lower_name.endswith("quarterly") or lower_name.endswith("quaterly")
        ):
            df = getattr(symbol, attr_name, None)
            if isinstance(df, pd.DataFrame) and not df.empty:
                tables.append(df)
            # else:
            #     self.logger.debug("Quarterly table '%s' not found or empty", attr_name)
    return tables

In [6]:
storage.get_table("balance_sheet_quarterly")

,symbol,fiscal_date_ending,reported_currency,total_assets,total_current_assets,cash_and_cash_equivalents,cash_and_short_term_investments,inventory,total_non_current_assets,property_plant_equipment,...,current_net_receivables,intangible_assets,intangible_assets_excluding_goodwill,goodwill,long_term_investments,short_term_investments,total_liabilities,total_current_liabilities,total_shareholder_equity,common_stock_shares_outstanding
0,NVDA,2025-07-31,USD,1.407400e+11,1.022190e+11,1.163900e+10,1.163900e+10,1.496200e+10,3.852100e+10,NaN,...,2.780800e+10,755000000.0,755000000.0,5.755000e+09,3.799000e+09,4.515200e+10,4.060900e+10,2.425700e+10,1.001310e+11,24532000000
1,NVDA,2025-04-30,USD,1.252540e+11,8.993500e+10,1.523400e+10,1.523400e+10,1.133300e+10,3.531900e+10,NaN,...,2.213200e+10,769000000.0,769000000.0,5.498000e+09,3.240000e+09,3.745700e+10,4.141100e+10,2.654200e+10,8.384300e+10,24611000000
2,NVDA,2025-01-31,USD,1.116010e+11,8.012600e+10,8.589000e+09,8.589000e+09,1.008000e+10,3.147500e+10,NaN,...,2.306500e+10,807000000.0,807000000.0,5.188000e+09,3.387000e+09,3.462100e+10,3.227400e+10,1.804700e+10,7.932700e+10,24706000000
3,NVDA,2024-10-31,USD,9.601300e+10,6.764000e+10,9.107000e+09,9.107000e+09,7.654000e+09,2.837300e+10,NaN,...,1.769300e+10,838000000.0,838000000.0,4.724000e+09,2.237000e+09,2.938000e+10,3.011400e+10,1.647900e+10,6.589900e+10,24774000000
4,NVDA,2024-07-31,USD,8.522700e+10,5.963300e+10,8.563000e+09,8.563000e+09,6.675000e+09,2.559400e+10,NaN,...,1.413200e+10,952000000.0,952000000.0,4.622000e+09,1.819000e+09,2.623700e+10,2.707000e+10,1.396900e+10,5.815700e+10,24848000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,NVDA,2006-07-31,USD,2.184000e+09,1.726000e+09,3.880000e+08,3.880000e+08,3.790000e+08,NaN,177144000.0,...,4.600000e+08,29000000.0,29000000.0,2.040000e+08,NaN,4.630000e+08,5.290000e+08,5.050000e+08,1.654548e+09,57840000000
77,NVDA,2006-04-30,USD,2.161453e+09,1.722627e+09,4.877630e+08,4.877630e+08,3.463660e+08,4.388260e+08,180206000.0,...,3.912700e+08,22989000.0,22989000.0,2.046030e+08,NaN,4.672900e+08,5.528020e+08,5.343860e+08,1.608651e+09,23422200000
78,NVDA,2006-01-31,USD,1.915299e+09,1.548932e+09,5.517560e+08,5.517560e+08,2.547920e+08,3.663670e+08,178152000.0,...,3.181860e+08,160738000.0,160738000.0,1.453170e+08,NaN,3.984180e+08,4.575430e+08,4.386590e+08,1.457756e+09,22312000000
79,NVDA,2005-10-31,USD,1.805508e+09,1.449976e+09,3.643080e+08,3.643080e+08,2.848470e+08,3.555320e+08,178505000.0,...,3.517370e+08,152314000.0,152314000.0,1.330000e+08,NaN,4.218700e+08,4.560670e+08,4.271670e+08,1.349441e+09,22000419800


In [3]:
storage.balance_sheet_quarterly

,symbol,fiscal_date_ending,reported_currency,total_assets,total_current_assets,cash_and_cash_equivalents,cash_and_short_term_investments,inventory,total_non_current_assets,property_plant_equipment,...,current_net_receivables,intangible_assets,intangible_assets_excluding_goodwill,goodwill,long_term_investments,short_term_investments,total_liabilities,total_current_liabilities,total_shareholder_equity,common_stock_shares_outstanding
0,NVDA,2025-07-31,USD,1.407400e+11,1.022190e+11,1.163900e+10,1.163900e+10,1.496200e+10,3.852100e+10,NaN,...,2.780800e+10,755000000.0,755000000.0,5.755000e+09,3.799000e+09,4.515200e+10,4.060900e+10,2.425700e+10,1.001310e+11,24532000000
1,NVDA,2025-04-30,USD,1.252540e+11,8.993500e+10,1.523400e+10,1.523400e+10,1.133300e+10,3.531900e+10,NaN,...,2.213200e+10,769000000.0,769000000.0,5.498000e+09,3.240000e+09,3.745700e+10,4.141100e+10,2.654200e+10,8.384300e+10,24611000000
2,NVDA,2025-01-31,USD,1.116010e+11,8.012600e+10,8.589000e+09,8.589000e+09,1.008000e+10,3.147500e+10,NaN,...,2.306500e+10,807000000.0,807000000.0,5.188000e+09,3.387000e+09,3.462100e+10,3.227400e+10,1.804700e+10,7.932700e+10,24706000000
3,NVDA,2024-10-31,USD,9.601300e+10,6.764000e+10,9.107000e+09,9.107000e+09,7.654000e+09,2.837300e+10,NaN,...,1.769300e+10,838000000.0,838000000.0,4.724000e+09,2.237000e+09,2.938000e+10,3.011400e+10,1.647900e+10,6.589900e+10,24774000000
4,NVDA,2024-07-31,USD,8.522700e+10,5.963300e+10,8.563000e+09,8.563000e+09,6.675000e+09,2.559400e+10,NaN,...,1.413200e+10,952000000.0,952000000.0,4.622000e+09,1.819000e+09,2.623700e+10,2.707000e+10,1.396900e+10,5.815700e+10,24848000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,NVDA,2006-07-31,USD,2.184000e+09,1.726000e+09,3.880000e+08,3.880000e+08,3.790000e+08,NaN,177144000.0,...,4.600000e+08,29000000.0,29000000.0,2.040000e+08,NaN,4.630000e+08,5.290000e+08,5.050000e+08,1.654548e+09,57840000000
77,NVDA,2006-04-30,USD,2.161453e+09,1.722627e+09,4.877630e+08,4.877630e+08,3.463660e+08,4.388260e+08,180206000.0,...,3.912700e+08,22989000.0,22989000.0,2.046030e+08,NaN,4.672900e+08,5.528020e+08,5.343860e+08,1.608651e+09,23422200000
78,NVDA,2006-01-31,USD,1.915299e+09,1.548932e+09,5.517560e+08,5.517560e+08,2.547920e+08,3.663670e+08,178152000.0,...,3.181860e+08,160738000.0,160738000.0,1.453170e+08,NaN,3.984180e+08,4.575430e+08,4.386590e+08,1.457756e+09,22312000000
79,NVDA,2005-10-31,USD,1.805508e+09,1.449976e+09,3.643080e+08,3.643080e+08,2.848470e+08,3.555320e+08,178505000.0,...,3.517370e+08,152314000.0,152314000.0,1.330000e+08,NaN,4.218700e+08,4.560670e+08,4.271670e+08,1.349441e+09,22000419800


In [2]:
from analyst.financial_analyst.financial_analyst import FinancialAnalyst
fin_analyst = FinancialAnalyst(storage)
fin_analyst.run_analysis()

2025-09-21 14:39:32,109 - analyst.financial_analyst.financial_analyst - ERROR - financial_analyst.py:190 - Failed to read PDF /home/bandee/projects/marketIntelligence/infrastructure/databases/rag_knowledge_base/financial_analyst/Breaking Down the Balance Sheet.pdf: [Errno 2] No such file or directory: '/home/bandee/projects/marketIntelligence/infrastructure/databases/rag_knowledge_base/financial_analyst/Breaking Down the Balance Sheet.pdf'
2025-09-21 14:39:32,110 - analyst.financial_analyst.financial_analyst - WARNING - financial_analyst.py:204 - No text extracted from ./infrastructure/databases/rag_knowledge_base/financial_analyst/Breaking Down the Balance Sheet.pdf for metric balance_sheet
2025-09-21 14:39:32,111 - analyst.financial_analyst.financial_analyst - ERROR - financial_analyst.py:190 - Failed to read PDF /home/bandee/projects/marketIntelligence/infrastructure/databases/rag_knowledge_base/financial_analyst/Income Statement_ How to Read and Use It.pdf: [Errno 2] No such file o

({'balance_sheet': None, 'income_statement': None, 'cash_flow': None},
 None,
 {'balance_sheet': 'No data available',
  'income_statement': 'No data available',
  'cash_flow': 'No data available'})

In [3]:
analyst = QuantitativeAnalyst(storage)
analyst.run_analysis()

2025-09-21 14:25:57,944 - httpx - INFO - _client.py:1025 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-21 14:26:00,184 - httpx - INFO - _client.py:1025 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-21 14:26:04,371 - httpx - INFO - _client.py:1025 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-21 14:26:08,473 - httpx - INFO - _client.py:1025 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-21 14:26:12,642 - httpx - INFO - _client.py:1025 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-21 14:26:16,311 - httpx - INFO - _client.py:1025 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-21 14:26:19,782 - httpx - INFO - _client.py:1025 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-21 14:26:22,685 - h

{'symbol': 'NVDA',
 'epoch': 1756418400,
 'last_date': '2025-08-29 00:00:00',
 'num_data_points': 251,
 'indicators_evaluated': ['obv',
  'vwap',
  'macd',
  'rsi',
  'stochastic_oscillator',
  'bollinger_bands',
  'sma',
  'ema'],
 'analysis_run_timestamp': 1758457582,
 'analysis_run_datetime': '2025-09-21 14:26:22',
 'warnings': [],
 'errors': [],
 'indicator_scores': {'obv': 0.6,
  'vwap': 1.0,
  'macd': 1.0,
  'rsi': 1.0,
  'stochastic_oscillator': 1.0,
  'bollinger_bands': 1.0,
  'sma': 0.8,
  'ema': 1.0},
 'aggregate_score': 0.925,
 'explanations': {'obv': 'The OBV values over the past year show a generally upward trend with some fluctuations, indicating increasing volume flow associated with rising prices. The OBV line appears to be trending higher, suggesting accumulation by "smart money" and potential bullish momentum. \n\nGiven the overall positive slope and the absence of significant divergences or sharp reversals, the company seems to be in a phase of accumulation, which co

TypeError: FinancialAnalyst.__init__() missing 1 required positional argument: 'article_paths'